In [2]:
# Setup
import pandas as pd

raw_data = pd.read_csv("https://raw.githubusercontent.com/bear-revels/immo-eliza-scraping-Python_Pricers/main/data/all_property_details.csv")

In [ ]:
raw_data.head()

In [ ]:
raw_data.notnull().sum()

In [ ]:
raw_data.isna().sum() 

In [ ]:
raw_data[raw_data['Street'].str.contains('�', na=False)]

In [24]:
raw_data.columns

Index(['ID', 'Street', 'HouseNumber', 'Box', 'Floor', 'City', 'PostalCode',
       'Region', 'District', 'Province', 'PropertyType', 'PropertySubType',
       'Price', 'SaleType', 'ConstructionYear', 'BedroomCount', 'LivingArea',
       'KitchenType', 'Furnished', 'Fireplace', 'Terrace', 'TerraceArea',
       'Garden', 'GardenArea', 'Facades', 'SwimmingPool', 'Condition',
       'EPCScore', 'Latitude', 'Longitude', 'PropertyUrl', 'Property url'],
      dtype='object')

In [8]:
raw_data['PropertySubType'].unique()


array(['HOUSE', 'VILLA', 'MANSION', 'APARTMENT_BLOCK',
       'MIXED_USE_BUILDING', 'EXCEPTIONAL_PROPERTY', 'TOWN_HOUSE',
       'MANOR_HOUSE', 'BUNGALOW', 'COUNTRY_COTTAGE', 'FARMHOUSE',
       'CASTLE', 'OTHER_PROPERTY', 'CHALET', 'APARTMENT', 'PENTHOUSE',
       'DUPLEX', 'FLAT_STUDIO', 'SERVICE_FLAT', 'TRIPLEX', 'KOT',
       'GROUND_FLOOR', 'LOFT'], dtype=object)

In [10]:
raw_data['District'].unique()

array(['Nivelles', 'Dinant', 'Brussels', 'Gent', 'Dendermonde',
       'Mechelen', 'Li�ge', 'Leuven', 'Oudenaarde', 'Marche-en-Famenne',
       'Antwerp', 'Sint-Niklaas', 'Arlon', 'Neufch�teau', 'Waremme',
       'Verviers', 'Huy', 'Halle-Vilvoorde', 'Bastogne', 'Aalst',
       'Oostend', 'Soignies', 'Brugge', 'Tielt', 'Charleroi', 'Thuin',
       'Virton', 'Kortrijk', 'Mons', 'Turnhout', 'Tournai', 'Namur',
       'Roeselare', 'Mouscron', 'Maaseik', 'Ath', 'Tongeren', 'Eeklo',
       'Veurne', 'Hasselt', 'Philippeville', 'Ieper', 'Diksmuide'],
      dtype=object)